In [1]:
from sklearn.cross_validation import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_scoreore, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
def plot_confusion_matrix(cm):
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.set_title('Confusion Matrix')
    fig.colorbar(im)

    target_names = ['not survived', 'survived']

    tick_marks = np.arange(len(target_names))
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(target_names, rotation=45)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(target_names)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')
    fig.tight_layout()

In [3]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [4]:
df_train.drop('PassengerId', axis=1, inplace=True)
df_train.drop('Name', axis=1, inplace=True)
df_train.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38,1,0,PC 17599,71.2833,C85,C


In [5]:
#df_train.groupby([df_train.Pclass, df_train.Sex]).Age.mean()

In [6]:
def fill_fare(df):
    df['Fare'].fillna(0, inplace=True)
    df['FareFill'] = df.Fare
    df.FareFill[(df.Fare == 0) & (df.Pclass == 1)] = 86
    df.FareFill[(df.Fare == 0) & (df.Pclass == 2)] = 21
    df.FareFill[(df.Fare == 0) & (df.Pclass == 3)] = 13
    df.FareFill = df.FareFill.apply(lambda f:np.log(f))
    return df

In [7]:
def fill_age(df):
    df['AgeFill'] = df.Age
    df.AgeFill[df.Age.isnull() & (df.Sex == 'male') & (df.Pclass == 1)] = 41
    df.AgeFill[df.Age.isnull() & (df.Sex == 'male') & (df.Pclass == 2)] = 30
    df.AgeFill[df.Age.isnull() & (df.Sex == 'male') & (df.Pclass == 3)] = 26
    df.AgeFill[df.Age.isnull() & (df.Sex == 'female') & (df.Pclass == 1)] = 34
    df.AgeFill[df.Age.isnull() & (df.Sex == 'female') & (df.Pclass == 2)] = 28
    df.AgeFill[df.Age.isnull() & (df.Sex == 'female') & (df.Pclass == 3)] = 21
    return df

In [8]:
def extract_embarked(df):
    df['is_cherbourg'] = 0
    df.is_cherbourg = df.Embarked.apply(lambda e: 1 if e == 'C' else 0)
    return df

In [9]:
def extract_pclass(df):
    pclass_new = pd.get_dummies(df.Pclass)
    pclass_new.rename(columns=lambda x: 'pclass' + "_" + str(x), inplace=True)
    df = df.join(pclass_new)
    return df

In [17]:
def classfy_cabin(cabin_str):
    if cabin_str == 'X':
        return 'X'
    try:
        deck = cabin_str[0]
    except:
        return 'X'

def extract_cabin(df):
    df['CCabin'] = df.Cabin
    df.CCabin[df.Cabin.isnull()] = 'X'
    df['hasCabin'] = df.CCabin.apply(lambda c: 0 if c == 'X' else 1)
    return df
    #dummies = pd.get_dummies(df.CCabin.apply(lambda c:classfy_cabin(c)))
    #dummies.drop('X', axis=1, inplace=True)
    #dummies.rename(columns=lambda x: 'cabin' + "_" + str(x), inplace=True)
    #return df.join(dummies)

In [18]:
def convert_sex(df):
    df['Gender'] = df.Sex.apply(lambda s: 0 if s == 'male' else 1)
    return df

In [19]:
def extract_feature(df):
    df = fill_age(df)
    #df = extract_embarked(df)
    df = extract_pclass(df)
    df = convert_sex(df)
    df = fill_fare(df)
    df = extract_cabin(df)
    cols = df.columns
    drop_cols = set(cols).intersection(set(['PassengerId', 'Name', 'SibSp', 'Ticket', 'Fare', 'Pclass', 'Survived', 'Parch', 'Sex', 'Age', 'Ticket', 'Cabin', 'Embarked', 'CCabin']))
    return df.drop(drop_cols, axis=1)

In [20]:
def get_classifier():
    clf = LogisticRegression(C=100, penalty='l2', tol=0.01)
    #clf = RandomForestClassifier()
    #clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=2)
    return clf

In [21]:
def calc_classifier(df):
    X_train = extract_feature(df)
    y_train = df['Survived']
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=0)
    print('Num of Training Samples: {}'.format(len(X_train)))
    print('Num of Validation Samples: {}'.format(len(X_val)))
    
    clf = get_classifier()
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_val_pred = clf.predict(X_val)
    print('Accuracy on Training Set: {:.3f}'.format(accuracy_score(y_train, y_train_pred)))
    print('Accuracy on Validation Set: {:.3f}'.format(accuracy_score(y_val, y_val_pred)))
    cm = confusion_matrix(y_val, y_val_pred)
    return clf

In [22]:
def cross_val(X, y, K, random_state=0, clf=None, ):
    if clf is None:
        clf = get_classifier()
    cv = KFold(len(y), K, shuffle=True, random_state=random_state)
    scores = cross_val_score(clf, X, y, cv=cv)
    print('Scores:', scores)
    print('Mean Score: {0:.3f} (+/-{1:.3f})'.format(scores.mean(), scores.std()*2))
    return scores

In [23]:
X_train = extract_feature(df_train)
y_train = df_train.Survived
#cross_val(X_train, y_train, 5, clf=LogisticRegression(C=100, penalty='l2', tol=0.01))
#cross_val(X_train, y_train, 5, clf=LogisticRegression(C=10, penalty='l2', tol=0.01))
#cross_val(X_train, y_train, 5, clf=LogisticRegression(C=1, penalty='l2', tol=0.01))
#cross_val(X_train, y_train, 5, clf=LogisticRegression(C=100, penalty='l1', tol=0.01))
#cross_val(X_train, y_train, 5, clf=LogisticRegression(C=10, penalty='l1', tol=0.01))
cross_val(X_train, y_train, 5, clf=LogisticRegression(C=0.1, penalty='l2', tol=0.01))

('Scores:', array([ 0.79888268,  0.78651685,  0.79213483,  0.80898876,  0.75842697]))
Mean Score: 0.789 (+/-0.034)


array([ 0.79888268,  0.78651685,  0.79213483,  0.80898876,  0.75842697])

In [24]:
clf = calc_classifier(df_train)

Num of Training Samples: 712
Num of Validation Samples: 179
Accuracy on Training Set: 0.812
Accuracy on Validation Set: 0.816


In [25]:
X_train.head(5)

,AgeFill,pclass_1,pclass_2,pclass_3,Gender,FareFill,hasCabin
0,22,0,0,1,0,1.981001,0
1,38,1,0,0,1,4.266662,1
2,26,0,0,1,1,2.070022,0
3,35,1,0,0,1,3.972177,1
4,35,0,0,1,0,2.085672,0


In [103]:
Y = extract_feature(df_test)
df_test['Survived'] = clf.predict(Y)
submit_data = df_test[['PassengerId', 'Survived']]

Index([u'PassengerId', u'Pclass', u'Name', u'Sex', u'Age', u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked', u'AgeFill', u'pclass_1', u'pclass_2', u'pclass_3', u'Gender', u'FareFill', u'CCabin', u'cabin_A', u'cabin_B', u'cabin_C'], dtype='object')
set(['Fare', 'Name', 'Embarked', 'Age', 'Parch', 'Pclass', 'Sex', 'CCabin', 'SibSp', 'PassengerId', 'Ticket', 'Cabin'])


In [104]:
Y.head()

,AgeFill,pclass_1,pclass_2,pclass_3,Gender,FareFill,cabin_A,cabin_B,cabin_C
0,34.5,0,0,1,0,2.057860,0,0,0
1,47.0,0,0,1,1,1.945910,0,0,0
2,62.0,0,1,0,0,2.270836,0,0,0
3,27.0,0,0,1,0,2.159003,0,0,0
4,22.0,0,0,1,1,2.508582,0,0,0


In [105]:
submit_data.to_csv('./submit_simple_log2_logfare_emb.csv', index=False)